<br>

# Introdução


In [ ]:
#!pip3 install traquitanas --upgrade


In [ ]:
import pprint
import time

import pandas as pd
from my_driver import Driver
from paths import (
    adds_path,
    driver_path,
    input_path_down,
    input_path_tab_temp,
    logs_path,
)
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService
from traquitanas.scrapping import adds, gecko

# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import WebDriverWait
# input_path

<br>

## Driver


In [ ]:
# # Gecko
# gecko_path = gecko.get_path_geckodriver(driver_path)

# # Logs
# logs_filepath = logs_path / 'geckodriver.log'

# # Services
# service = FirefoxService(executable_path=gecko_path, log_path=logs_filepath)

# # Options
# options = FirefoxOptions()
# options.headless = False
# options.set_preference('intl.accept_languages', 'pt-BR, pt')
# options.set_preference('browser.download.folderList', 2)
# options.set_preference('browser.download.manager.showWhenStarting', False)
# options.set_preference('browser.download.dir', input_path_down.as_posix())
# options.set_preference(
#     'browser.helperApps.neverAsk.saveToDisk',
#     'application/octet-stream, application/pdf, application/vnd.ms-excel',
# )


# # Create Driver
# driver = webdriver.Firefox(service=service, options=options)
# driver.maximize_window()

# # Add Extension
# driver = adds.add_extension_xpath(driver, adds_path)


<br>

## Functions


In [ ]:
driver = Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,    
    my_download_path=input_path_down,
    verify_ssl=True,
)

In [ ]:
def get_fbds():
    """
    _summary_
    """
    URL = 'https://geo.fbds.org.br/SP/'
    driver.get(URL)
    time.sleep(5)


In [ ]:
def click_back():
    """
    Voltar

    :return: _description_
    :rtype: _type_
    """
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")
    back_xpath = content_xpath.find_element(
        By.XPATH, ".//*[@class='item folder folder-parent']"
    )
    back_xpath.click()
    time.sleep(1)
    return 0


In [ ]:
def click_download():
    """
    _summary_

    :return: _description_
    :rtype: _type_
    """
    content_xpath = driver.find_element(By.XPATH, "//*[@id='topbar']")
    down_xpath = content_xpath.find_element(By.XPATH, ".//*[@id='download']")
    down_xpath.click()
    time.sleep(1)
    return 0


In [ ]:
def get_dict(folder):
    """
    XPATH dos folders
    :param folder: _description_
    :type folder: _type_
    """
    # URL
    href_xpath = folder.find_element(By.XPATH, ".//a")
    href_value = href_xpath.get_attribute('href')
    # print(href_value)

    # Tipo
    icon_xpath = folder.find_element(
        By.XPATH, ".//a//span[@class='icon square']//img"
    )
    icon_value = icon_xpath.get_attribute('alt')
    # print(icon_value)

    # Nome
    label_xpath = folder.find_element(By.XPATH, ".//a//span[@class='label']")
    label_value = label_xpath.text
    # print(label_value)

    # Data
    date_xpath = folder.find_element(By.XPATH, ".//a//span[@class='date']")
    date_value = date_xpath.text
    # print(date_value)

    # Size
    size_xpath = folder.find_element(By.XPATH, ".//a//span[@class='size']")
    size_value = size_xpath.text
    # print(size_value)

    #
    dict_data = {
        'url': href_value,
        'type': icon_value,
        'name': label_value,
        'date': date_value,
        'size': size_value,
    }
    # print(dict_data)
    return dict_data


In [ ]:
def list_folders_h5ai():
    """
    Lista as Pastas

    :return: _description_
    :rtype: _type_
    """
    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    # Lista
    list_folders = content_xpath.find_elements(
        By.XPATH, ".//*[@class='item folder']"
    )

    list_data = []
    for folder in list_folders:
        dict_data = get_dict(folder)
        list_data.append(dict_data)
    return list_data


In [ ]:
def list_itens_h5ai():
    """
    _summary_

    :return: _description_
    :rtype: _type_
    """
    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    # Lista
    list_itens = content_xpath.find_elements(
        By.XPATH, ".//*[@class='item file']"
    )

    list_data = []
    for folder in list_itens:
        dict_data = get_dict(folder)
        list_data.append(dict_data)
    return list_data


In [ ]:
def find_folder_by_name(name, click=True):
    """
    _summary_

    :param name: _description_
    :type name: _type_
    :param click: _description_, defaults to True
    :type click: bool, optional
    :raises ValueError: _description_
    :return: _description_
    :rtype: _type_
    """
    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    try:
        foldername_xpath = content_xpath.find_element(
            By.XPATH, f".//span[text()='{name}']"
        )
        if click:
            foldername_xpath.click()
            time.sleep(1)
        return foldername_xpath
    except Exception as e:
        print(e)
        raise ValueError('A very specific bad thing happened')


# find_folder_by_name('ADAMANTINA')


In [ ]:
def save_dataframe(my_list, my_path):
    """
    _summary_

    :param my_list: _description_
    :type my_list: _type_
    :param my_path: _description_
    :type my_path: _type_
    """
    # Cria e Salva Tabela
    df = pd.DataFrame(my_list)
    df.to_csv(my_path / 'links.csv', encoding='utf-8', index=False)
    return 0


In [ ]:
def get_list_municipios():
    list_subfolders = list_folders_h5ai()
    print(f'São  {len(list_subfolders)} pastas')
    pprint.pprint(list_subfolders[:5])

    list_municipios = [i['name'] for i in list_subfolders]
    list_municipios.sort()
    print(f'São {len(list_municipios)} municípios')
    pprint.pprint(list_municipios[:5])
    return list_municipios


<br>

# Run


<br>

## Abordagem 1


In [ ]:
# Driver
get_fbds()


In [ ]:
# Initial Parameters
list_all_files = []

# Parametros
wait_seconds = 3


def loop():
    """
    Lista todos os arquivos recursivamente
    Lista todas as pastas e:
    - se Nº Pastas > 0, entra na pasta e... repete;
    - se Nº Pastas = 0, retorna pra pasta anterior;
    """

    # Lista arquivos
    list_files = list_itens_h5ai()

    # Se tem Arquivos, junta na lista geral
    if len(list_files) > 0:
        list_all_files.extend(list_files)
        print(f' > Lista geral está com: {len(list_all_files)} arquivos')
    else:
        print(f' > Pasta sem arquivos')

    # Lista Pastas
    _list_subfolders = list_folders_h5ai()

    if len(_list_subfolders) > 0:
        for subfolder in _list_subfolders:
            # Parâmetros
            subfolder_name = subfolder['name']
            print(f'.\\{subfolder_name}')

            # Entra na Pasta
            find_folder_by_name(subfolder_name)

            # Recursive
            loop()

            # Cria e Salva Tabela
            save_dataframe(list_all_files, input_path_tab_temp)

            # Sai da Pasta
            click_back()
            time.sleep(wait_seconds)
    else:
        print(f' > Pasta sem subpastas...')


In [ ]:
list_municipios = get_list_municipios()


In [ ]:
# Lista Pastas
list_folders = list_folders_h5ai()

for folder in list_folders:
    # Parâmetros
    nome_municipio = folder['name']

    if nome_municipio in list_municipios:
        # Prints
        print(f'Processando letra {nome_municipio}')

        # Entra na Pasta
        find_folder_by_name(nome_municipio)

        # Recursive
        loop()

        # Sai da Pasta
        click_back()
        time.sleep(wait_seconds)

# Fim
print('Fim')


<br>

## Abordagem 2


In [ ]:
# Driver
get_fbds()


In [ ]:
# Lista dos Município
list_municipios = get_list_municipios()
list_municipios[50:60]


In [ ]:
# Action
a = ActionChains(driver)

# Conteúdo Principal
content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

# Lista
list_folders = content_xpath.find_elements(
    By.XPATH,
    ".//*[@class='item folder' or @class='item folder selected' or @class='item folder folder-parent']",
)

# Results
print(f'São {len(list_folders)} pastas')
pprint.pprint(list_folders[:2])


In [ ]:
# list_folders = list_folders[:3]


In [ ]:
list_data = []

for count, folder in enumerate(list_folders):
    #
    infos = get_dict(folder)
    nome_municipio = infos['name']

    # ddddd
    if nome_municipio in list_municipios:
        # Prints
        print(f'Processando letra {nome_municipio}')

        # URL
        href_xpath = folder.find_element(By.XPATH, ".//a")
        href_value = href_xpath.get_attribute('href')

        # Tipo
        icon_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='icon square']//img"
        )
        icon_value = icon_xpath.get_attribute('alt')

        # Nome
        label_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='label']"
        )
        label_value = label_xpath.text

        # Data
        date_xpath = folder.find_element(By.XPATH, ".//a//span[@class='date']")
        date_value = date_xpath.text

        # Size
        size_xpath = folder.find_element(By.XPATH, ".//a//span[@class='size']")
        size_value = size_xpath.text

        # Selector
        selector_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='selector']"
        )

        # Dictionary
        dict_data = {
            'url': href_value,
            'type': icon_value,
            'name': label_value,
            'date': date_value,
            'size': size_value,
        }
        list_data.append(dict_data)

        # Vai pro Elemento
        print(f'inicio de download para {href_value}')
        time.sleep(1)

        # Xpath do Ícone Anterior e move pra lá
        folder_anterior = list_folders[count - 1]
        icon_xpath_anterior = folder_anterior.find_element(
            By.XPATH, ".//a//span[@class='icon square']//img"
        )
        driver.execute_script(
            "return arguments[0].scrollIntoView();", icon_xpath_anterior
        )
        time.sleep(1)

        # Passa o Mouse no Ícone da Pasta Atual
        a.move_to_element(icon_xpath).perform()
        time.sleep(3)

        # Passa o Mouse no Href da Pasta Atual
        a.move_to_element(href_xpath).perform()
        time.sleep(1)

        # Passa o Mouse no Selector da Pasta Atual e Clica!
        a.move_to_element(selector_xpath).click().perform()
        time.sleep(4)

        # Faz Download
        click_download()
        time.sleep(4)

        # Passa o Mouse no Href da Pasta Atual
        a.move_to_element(icon_xpath).perform()
        time.sleep(2)

        # Passa o Mouse no Selector da Pasta Atual e Clica!
        a.move_to_element(selector_xpath).click().perform()
        time.sleep(2)

# dddd
list_data


<br>

## Pastas Atuais


In [ ]:
# Lista todos os arquivos tar e, consequentemente, os municípios com download realizado
list_downloads = list(input_path_down.rglob('*.tar'))
list_downloads = [f.stem for f in list_downloads]
list_downloads = list(set(list_downloads))
list_downloads.sort()
list_downloads

# Resultados: Nº downloads
print(f'São {len(list_downloads)} downloads')
pprint.pprint(list_downloads[:5])


In [ ]:
# Defini Municípios Faltantes
list_municipios = list(set(list_municipios).difference(list_downloads))
list_municipios.sort()
list_municipios
